# Restart the MDAC

In [1]:
%matplotlib inline
import local_broom

Added 'quench/' and 'quench/libraries' for easy importing.


In [2]:
# Automagically reload modules when they are edited
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time
from qcodes import Station, Instrument
from tqdm.notebook import tqdm

from monty import Monty
import MDAC
import may.dots as dots
from may.custom_devices import connect_to_gb, newSiDot


In [4]:
try:
    mdac = Instrument.find_instrument("mdac")
    mdac.close()
except KeyError:
    print('Attempting to remove instrument with name mdac. Does not exist')

mdac = MDAC.MDAC('mdac', 'ASRL11::INSTR')

gb_control_si = connect_to_gb(mdac)  # Create our custom MDAC mappings
si = newSiDot(mdac)

Attempting to remove instrument with name mdac. Does not exist
Connected to: QNL Sydney MDAC (serial:0x32353532353551040030003E, firmware:1.8) in 4.68s
Loading GB driver and connecting
Manually Open Breakout_box_top_ch15_LCB
Manually Open Breakout_box_bot_ch09_RCB
Manually Open Breakout_box_bot_ch17_RG
Manually Open Breakout_box_top_ch17_ResB
Manually Open Breakout_box_top_ch19_P3
Manually Open Breakout_box_top_ch13_P2
Manually Open Breakout_box_top_ch04_P1
Manually Open Breakout_box_top_ch02_SETB
Manually Open Breakout_box_bot_ch04_SRB
Manually Open Breakout_box_bot_ch06_SLB
Manually Open Breakout_box_top_ch10_ST
Manually Open Breakout_box_top_ch25_bias


In [15]:
disconnect = False
if disconnect:
    try:
        mdac = Instrument.find_instrument("mdac")
        mdac.close()
    except KeyError:
        print('Attempting to remove instrument with name mdac. Does not exist')

In [11]:
dots.get_all_voltages(mdac)

LCB:   0.0 V
RCB:   0.0 V
RG:    0.0 V
ResB:  0.0 V
P3:    0.0 V
P2:    0.0 V
P1:    0.0 V
SETB:  0.0 V
SRB:   0.0 V
SLB:   0.0 V
ST:    0.0 V
bias:  0.0 V
VSS1P8:      0.0 V
VSS1P0:      0.0 V
VDD1P8:      1.800005 V
VDD1P0:      0.999996 V
VDD1P8_ANA:  1.8 V
BGN1P0:      0.0 V
BGN1P8:      0.0 V
BGP1P0:      0.0 V
BGP1P8:      0.0 V
RST:         1.799996 V
MOSI:        0.0 V
SCLK:        0.0 V
SS_N:        1.8 V
APBCLK:      0.0 V
VICL:        3.500004 V
VLFG:        0.0 V
VHFG:        0.0 V


### Powerdown

In [ ]:
# Power everything off

si.LCB(0)
si.RCB(0)
si.SRB(0)
si.SLB(0)
si.ST(0)
si.bias(0)
si.P1(0)
si.P2(0)

gb_control_si.VICL(0)

In [ ]:
gb_control_si.disable_all_gates()

In [ ]:
gb_control_si.power_down()

### Power up

See `may/power_failure.py` for where these methods are copied from

In [7]:
def output_changer(gate, microd = 'open', smc = 'open', dac_output = 'open', ground = 'open', bus = 'open', microd_connection = False, smc_connection = False, float_gate = False, ground_gate = False, do_print=True):
    '''Changes the output of any connection to the MDAC, user specified.'''
    if microd_connection:
        gate.source.smc('open') 
        gate.source.bus('open')
        gate.source.dac_output('close')
        gate.source.gnd('open')
        gate.source.microd('close')
    if smc_connection:
        gate.source.smc('close') 
        gate.source.bus('open')
        gate.source.dac_output('close')
        gate.source.gnd('open')
        gate.source.microd('open')
    if float_gate:
        gate.source.smc('open') 
        gate.source.bus('open')
        gate.source.dac_output('open')
        gate.source.gnd('open')
        gate.source.microd('open')
    if ground_gate:
        gate.source.smc('open') 
        gate.source.bus('open')
        gate.source.dac_output('open')
        gate.source.gnd('close')
        gate.source.microd('open')
    else:
        gate.source.smc(smc) 
        gate.source.bus(bus)
        gate.source.dac_output(dac_output)
        gate.source.gnd(ground)
        gate.source.microd(microd)
    if do_print:
        print(' NAME:      ', gate.name, '\n',
        'CHANNEL:   ', gate.source.name, '\n',
        'ground:    ', gate.source.gnd(),'\n',
        'microd:    ', gate.source.microd(),'\n',
        'smc:       ', gate.source.smc(),'\n',
        'dac_output:', gate.source.dac_output(),'\n',
        'bus:       ', gate.source.bus(),'\n',
        'voltage:   ', gate.source.voltage())

In [8]:

# The MDAC connections will reset. Uncomment these to make MDAC connections good

# GB - MDAC microd connections
output_changer(gb_control_si.VDD1P8, microd = 'close', smc = 'open', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(gb_control_si.VDD1P0, microd = 'close', smc = 'open', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(gb_control_si.VSS1P8, microd = 'close', smc = 'open', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(gb_control_si.VSS1P0, microd = 'close', smc = 'open', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(gb_control_si.VDD1P8_ANA, microd = 'close', smc = 'open', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(gb_control_si.BGP1P8, microd = 'close', smc = 'open', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(gb_control_si.BGP1P0, microd = 'close', smc = 'open', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(gb_control_si.BGN1P8, microd = 'close', smc = 'open', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(gb_control_si.BGN1P0, microd = 'close', smc = 'open', \
                dac_output = 'close', bus = 'open', ground = 'open')
    
# GB - MDAC SMC connections
output_changer(gb_control_si.RST_N, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(gb_control_si.MOSI, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(gb_control_si.SCLK, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(gb_control_si.SS_N, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(gb_control_si.APBCLK, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(gb_control_si.VICL, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(gb_control_si.VLFG, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(gb_control_si.VHFG, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
    
# GB test channels - MDAC SMC connections
output_changer(gb_control_si.MISO, microd = 'open', smc = 'close', \
                dac_output = 'open', bus = 'open', ground = 'open')
output_changer(gb_control_si.DTEST_1, microd = 'open', smc = 'close', \
                dac_output = 'open', bus = 'open', ground = 'open')
output_changer(gb_control_si.DTEST_2, microd = 'open', smc = 'close', \
                dac_output = 'open', bus = 'open', ground = 'open')
output_changer(gb_control_si.ATEST, microd = 'open', smc = 'close', \
                dac_output = 'open', bus = 'open', ground = 'open')
output_changer(gb_control_si.TMODE, microd = 'open', smc = 'close', \
                dac_output = 'open', bus = 'open', ground = 'open')
    
# Si channels
output_changer(si.LCB, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(si.RCB, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(si.RG, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(si.ResB, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(si.P3, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(si.P2, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(si.P1, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(si.SETB, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(si.SRB, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(si.SLB, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(si.ST, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')
output_changer(si.bias, microd = 'open', smc = 'close', \
                dac_output = 'close', bus = 'open', ground = 'open')

 NAME:       VDD1P8 
 CHANNEL:    mdac_chan34 
 ground:     open 
 microd:     close 
 smc:        open 
 dac_output: close 
 bus:        open 
 voltage:    0.0
 NAME:       VDD1P0 
 CHANNEL:    mdac_chan32 
 ground:     open 
 microd:     close 
 smc:        open 
 dac_output: close 
 bus:        open 
 voltage:    0.0
 NAME:       VSS1P8 
 CHANNEL:    mdac_chan2 
 ground:     open 
 microd:     close 
 smc:        open 
 dac_output: close 
 bus:        open 
 voltage:    0.0
 NAME:       VSS1P0 
 CHANNEL:    mdac_chan36 
 ground:     open 
 microd:     close 
 smc:        open 
 dac_output: close 
 bus:        open 
 voltage:    0.0
 NAME:       VDD1P8_ANA 
 CHANNEL:    mdac_chan29 
 ground:     open 
 microd:     close 
 smc:        open 
 dac_output: close 
 bus:        open 
 voltage:    0.0
 NAME:       BGP1P8 
 CHANNEL:    mdac_chan45 
 ground:     open 
 microd:     close 
 smc:        open 
 dac_output: close 
 bus:        open 
 voltage:    0.0
 NAME:       BGP1P0 
 CHANNEL: 

In [10]:
# Setup gooseberry

#connect_APBCLK_to_mdac()
#time.sleep(2) 
#en_charge_lock()

print('Ramping P gates up, J down to preserve electron count...')
# si.P1(2.1)
# si.P2(2.1)
mdac.ch05.ramp(0.0, 0.1)
mdac.ch05.block()
time.sleep(5)

print('Disabling GB gates...')
gb_control_si.disable_all_gates()

print('Restarting GB...')
gb_control_si.power_down()
time.sleep(1)
gb_control_si.power_up()
time.sleep(1)
gb_control_si.reset()

print('Ramping J1...')
gb_control_si.enable_gate(20)  # close GB gate to J1
# gb_control_si.enable_multiple_gates([18, 20])

time.sleep(1)
mdac.ch05.ramp(3.5, 0.1) # VICL, has a 2.5:1 divider
mdac.ch05.block()

Ramping P gates up, J down to preserve electron count...
Disabling GB gates...
Restarting GB...
Ramping J1...


In [14]:
dots.get_all_voltages(mdac)

LCB:   0.150004 V
RCB:   0.150003 V
RG:    0.0 V
ResB:  0.0 V
P3:    0.0 V
P2:    0.0 V
P1:    0.0 V
SETB:  0.0 V
SRB:   0.900001 V
SLB:   0.899997 V
ST:    3.300003 V
bias:  0.0 V
VSS1P8:      0.0 V
VSS1P0:      0.0 V
VDD1P8:      1.800005 V
VDD1P0:      0.999996 V
VDD1P8_ANA:  1.8 V
BGN1P0:      0.0 V
BGN1P8:      0.0 V
BGP1P0:      0.0 V
BGP1P8:      0.0 V
RST:         1.799996 V
MOSI:        0.0 V
SCLK:        0.0 V
SS_N:        1.8 V
APBCLK:      0.0 V
VICL:        3.500004 V
VLFG:        0.0 V
VHFG:        0.0 V


In [13]:
# Return to nominal values
si.LCB(0.15)
si.RCB(0.15)
si.SLB(0.9)
si.SRB(0.9)
si.ST(3.3)